In [7]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

from counta_bot.utils.elastic_db import ElasticDB
#from utils.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

# news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
# wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

db = ElasticDB(elastic_port=PORT)

INFO:counta_bot.utils.elastic_db:Connecting to http://localhost:9200 
INFO:counta_bot.utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [8]:
### LOAD DATASETS ###
import json

# Note: Index into json.loads(object)[0] to extract dictionary object
args = [json.loads(ln)[0] for ln in open("../../data/processed_train_cmv.jsonl")]
topics = [json.loads(ln) for ln in open("../../data/claim_topics.jsonl")]
ex_retrieval = [json.loads(ln) for ln in open("../../data/wiki_doc_retrieved_from_op_train.jsonlist")]
ex_ranked = [json.loads(ln) for ln in open("../../data/selected_evidence.jsonl")]
conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]


In [9]:
### SAMPLE OUTPUT ###
import random
# TODOs: Replicate Output, Passage Evidence Retrieval
# TODOs: Replicate Output, Passage Ranking

_ = random.randint(0, 99)
ranked = ex_ranked[_]
retireval = ex_retrieval[_]

retireval

{'tid': 't3_3dn6d1',
 'retrieval_results': [{'retrieved_document_titles': ['European Union law',
    'Sex segregation',
    'Student rights in higher education',
    'Lakewood School District (Washington)',
    'Lakewood, Dallas'],
   'query': 'equal treatment power nature',
   'sentence': "CMV : Equal treatment and opportunities for all is dangerous for society in the long run , because it allows those who would n't naturally succeed and climb nature 's ladder to reach positions of leadership and power where they can do serious damage . "},
  {'retrieved_document_titles': ['Marriage Equality (Same Sex) Act 2013',
    'Recognition of same-sex unions in Australia',
    'History of same-sex marriage in Australia',
    'Detroit Tribune',
    'Anchor escapement'],
   'query': 'any act act equality equal',
   'sentence': "First off , i do n't think that any two people are exactly equal so any act act of equality is really a false show of compassion , although we are all imperfect . "},
  {'

In [10]:
# TODOs: News Data
# TODOs: Ranking
# TODOs: Ranking, Cosine
# TODOs: Research Evidence Retireval: Context Aware, Neural Retrieval
# TODOs: Stance
# TODOs: Target ADUs: Premises, Claims, discard non-ADUs, thus reducing noise over retreival 
# TODOs: Paralellise

# TERMS => Input Argument : args : arg
# TERMS => Argument Discourse Units : adu : adu_prem : adu_claim
# TERMS => Evidence : ev
# TERMS => Counter Evidence : counta_ev

In [11]:
### SUBJECT ARG ###
import random
import re

# Note: 340; Gender Equality
# Note: 991; Abortion

print(_)

claim = args[_]["title"]
arg = args[_]["argument"]
claim, arg

7


('Irony aside most people that post in are uneducated and changing their view on one point wont solve this overall problem',
 'There are some instances yes where controversial topics are discussed here and people are actually interested in hearing the opposite side. I consider myself one of those people and was initially driven to this subreddit in the hope that I would meet other people like myself that were posting here to educate themselves. The type of man that reads a book that argues against a position he already holds in the hopes that he can understand the entire issue in a great light.But far and away this is not the case. Some of the posts on here include logical fallacies even in the titles yesterdays atheism post used a strawman fallacy in the title to misrepresent their own position to make it easier to defend in a religion vs. athiesm debate. The OP if you read that post was by and large not interested in changing his opinion and this is true of the vast majority of posts

In [12]:
### NLP FUNCTIONS ###
import re

def sentences_segment(doc):
    return [i for i in re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', doc)]

def tokeniser(doc):
    return re.findall(r"\w+(?:'\w+)?|[^\w\s]", doc)

# Test Statements
tokeniser("hello, my name is Josh!")
sentences_segment("hello, my name is Josh! How are you doing today? I'm curious ... will this line seperate? I'm not so sure Dr. Evil")

['hello, my name is Josh! How are you doing today?',
 "I'm curious ... will this line seperate?",
 "I'm not so sure Dr. Evil"]

In [13]:
from counta_bot.detection.stance_classifier import sentence_stance, compare_stance

# TODOs: Implement Keyphrase Extraction and Tokenisation as a pre-processing step; Be mindful of interactive pipeline mode
# TODOs: Implement Semantic Search: https://www.elastic.co/blog/text-similarity-search-with-vectors-in-elasticsearch 
# TODOs: Fix Sentencizer
# TODOs: Tokeniser Function
# TODOs: Domain Restrict - polarising social and political debate (Class labelling); Note: currently, open-domain. Score Highly Polarised Discussions.
# TODOs: Bag of Topics, Concepts for Topic Labelling
# TODOs: News, Political, Sociology and 'Good', 'Positive' counter-evidence Knowledge Base.
# TODOs: Consider parsing and normalising knowledge; extracting core ADU arguments, premises, evidence and claims.
# TODOs: Sentence Segment Function
# TODOs: Semantic Retrieval ** 
# TODOs: Fix Sentence parsing
# TODOs: Parameterise Index-DB in use
# TODOs: Implement as a Class
# TODOs: Implement Logging
# TODOs: One Argument Loop (1 x # ADUs)
# TODOs: Filter ADUs; Strong and disputable arguments (Premise rakning, NLI, Argument Similarity (counter stance))
# TODOs: Reduce the size of the Argument using extractive summarisation
# TODOs: Reduce the size of the Argument Targeting Premises Only
# TODOs: Experiement with Query
# TODOs: Write to Database

### RETRIEVER ###
db = db
queries = []
retrieved_ev = []

def retrieved_evidence(arg):
    ad_units = sentences_segment(arg["argument"])

    results = []

    for adu in ad_units:
        toks = re.findall(r"\w+(?:'\w+)?|[^\w\s]", adu)

        if len(toks) <= 8:
            continue
    
        kp = arg["keyphrase"]

        if kp:
            query = ", ".join(i for i in kp)

            titles = [i["_source"]["document"]["title"] for i in db.search(query_=query, k=10)]
            evidence = [i["_source"]["document"]["text"] for i in db.search(query_=query, k=10)]

            results.append({
                "argument_discourse_unit": adu, 
                "retrieved_documents_titles": titles,
                "adu_keyphrases": [i for i in kp],
                "adu_stance": sentence_stance(adu, kp),
                "retrieved_evidence": evidence,
                "merged_evidence": ", ".join(ln for ln in evidence)
                })

        return results

retrieved_ev = []

for arg in args[0:100]:
    retrieved_ev.append(retrieved_evidence(arg))

INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.064s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.021s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.019s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.012s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.029s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.019s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.010s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.006s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.020s]
INFO:elastic_transport.transport:POST http://localhost:9200/*/_search [status:200 duration:0.014s]
INFO:elast

In [14]:
adu = 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural.'
ev_unit = 'In this essay, Mill argues the idea that the morality of an action can be judged by whether it is natural or unnatural.'
target = 'natural things'

stance = compare_stance(ev_unit, target)
stance

'CON'

In [18]:
from multiprocessing.pool import ThreadPool as Pool
from yake import KeywordExtractor
import tqdm.notebook as tqdm
import time
from summa import keywords
from tqdm import tqdm

### PASSAGE RANKING; KEYWORD OVERLAP ###
kw_extractor = KeywordExtractor(lan="en", n=3, top=5)

# TODOs: For each ADU, Rank Merged Evidence using Keyword Overlap and Filter for Contrasting Stance
# TODOs: Handel Multiple Keywords

def overlap_score(evidence_kp, adu_kp):
    score = 0
    # TODOs: Robust 'None' Handelling
    if adu_kp == None:
        return score
    # Split Keyphrase into components, scoring partial units as overlap
    else:
        for i in evidence_kp:
            for j in i.split():
                # Ensure string value, to enact .find
                if ", ".join([i for i in adu_kp]).find(j) != -1: score += 1
                
                else: continue
    
    return score

def calculate_overlap(merged_ev, adu_kp):

    for ev_unit in sentences_segment(merged_ev):
        toks = tokeniser(ev_unit)
        kp_overlap = 0
        
        if len(toks) <= 8: continue

        #ev_unit_kp = [i for i in keywords.keywords(ev_unit).split("\n")]
        ev_unit_kp = [i[0] for i in kw_extractor.extract_keywords(ev_unit)]

        if ev_unit_kp:
            kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
        
        else: ev_unit_kp = None
        yield ev_unit, ev_unit_kp, kp_overlap

# pool = Pool(8)
### RANK PASSAGES ###
def score_passages(ev_):   
    adu = ev_[0]["argument_discourse_unit"]
    adu_stance = ev_[0]["adu_stance"]
    merged_ev = ev_[0]["merged_evidence"] 
    adu_kp = ev_[0]["adu_keyphrases"]
    
    ### CALCULATE OVERLAP ###
    for ev_unit, ev_unit_kp, kp_overlap in calculate_overlap(merged_ev, adu_kp):
        target = adu_kp[0]

        compared_stace = compare_stance(ev_unit, target)
        if compared_stace != adu_stance:
            yield {
                "adu": adu,
                "adu_kp": adu_kp,
                "evidence_unit": ev_unit,
                "evidence_kps": ev_unit_kp,
                "overlap": kp_overlap,
                "evidence_stance": compare_stance(ev_unit, target),
                "adu_stance": adu_stance
            }

        else: continue

### SCORED EVIDENCE ###
def score_evidence(retrieved_evidence):
    for ev_ in retrieved_ev:
        yield [i for i in score_passages(ev_)]

### RANKED EVIDENCE ###
def rank_filter_counter_evidence(retireved_evidence, k=3):
    with tqdm(total=(len(retrieved_ev))) as pbar:
        for i in score_evidence(retrieved_ev):
            yield sorted(i, key=lambda y: y["overlap"], reverse=True)[0:k]

            pbar.update()


### SELECT TOP-K COUNTER-EVIDENCE ###
tic = time.time()
ranked_sorted_evidence = [i for i in rank_filter_counter_evidence(retrieved_ev)]
ranked_sorted_evidence
toc = time.time()

print(toc - tic)
# TIME 1:20M

100%|██████████| 100/100 [02:05<00:00,  1.26s/it]

125.6755211353302


In [19]:
ranked_sorted_evidence[0]

[{'adu': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural.',
  'adu_kp': ['unnatural', 'manmade things', 'natural processes'],
  'evidence_unit': 'The cause of illnesses are distinguished into categories of natural and unnatural illnesses.',
  'evidence_kps': ['distinguished into categories',
   'categories of natural',
   'natural and unnatural',
   'unnatural illnesses',
   'illnesses are distinguished'],
  'overlap': 4,
  'evidence_stance': 'PRO',
  'adu_stance': 'NEUTRAL'},
 {'adu': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural.',
  'adu_kp': ['unnatural', 'manmade things', 'natural processes'],
  'evidence_unit': 'Unnatural processes are logically conceivable but do not occur in nature.',
  'evidence_kps': ['Unnatural processes',
   'occur in nature',
   'processes are logically',
   'logically conceivable',
   'Unnatural'],
  '

In [ ]:
# # TODOs: Speed-up, Parrelleise, Yield
# def overlap_score(evidence_kp, adu_kp):
#     score = 0
    
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: score += 1
            
#             else: continue
    
#     return score

# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity'] 
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue
    
#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
    
#     adu_ev_overlap.append({
#         "adu": adu, 
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp, 
#         "kp_overlap": kp_overlap
        
#         })
        
# adu_ev_overlap

In [ ]:
# ### OVERLAP RANKED EVIDENCE ###

# adu_ev_overlap.sort(key=lambda y: y["kp_overlap"], reverse=True)
# adu_ev_overlap

# ### FILTER IRRELEVANT EVIDENCE ###
# overlapping = [i for i in adu_ev_overlap if i["kp_overlap"] !=0]

# len(adu_ev_overlap), len(overlapping)
# overlapping


In [ ]:
# ### ASSERT SAME STANCE ###
# from detection.stance_classifier import sentence_stance, compare_stance
#
# # TODOs: Ensure KPs Extracts are constrained to 1 unit
# opposing_stance = []
# for i in overlapping:
#     adu = i["adu"]
#     target = " ".join(i for i in i["adu_kp"])
#     ev_unit = i["ev_unit"]
#
#     ev_stance = compare_stance(ev_unit, ev_unit, target)
#     adu_stance = sentence_stance(adu, target)
#
#     if ev_stance != adu_stance:
#         opposing_stance.append((ev_unit, ev_stance, adu_stance))
#
#     else: continue
#
# opposing_stance

In [ ]:
### RANKING ###

# TODOs: Speed-up, Parrelleise, Yield
# ev_units = evidence
# adu_kp = extract_keyphrase(adu)

# adu_ev_overlap = []

# kp_1 = ['sex', 'relationship', 'opportunity'] 
# kp_2 = ['better sex']

# overlap_score(kp_2, kp_1)

# for ev_unit in evidence:
#     #print(ev_unit)
#     toks = tokeniser(ev_unit)

#     # Exprimental Value
#     if len(toks) <= 8:
#         continue
    
#     ev_unit_kp = extract_keyphrase(ev_unit)
#     kp_overlap = overlap_score(evidence_kp=ev_unit_kp, adu_kp=adu_kp)
    
#     adu_ev_overlap.append({
#         "adu": adu, 
#         "adu_kp": adu_kp,
#         "ev_unit": ev_unit,
#         "ev_unit_kp": ev_unit_kp, 
#         "kp_overlap": kp_overlap
        
#         })
        
# adu_ev_overlap


#rank_passages(retrieved_ev)

In [ ]:
# import spacy
# from spacy.matcher import PhraseMatcher
# from fuzzywuzzy import fuzz, process

# # TODOs: Package as a Module
# # TODOs: Handle Negation (Polarity shifters)
# # TODOs: Review Unsuperived Approach; Consider adveanced patterns and common-sence knowledge

# nlp = spacy.load("en_core_web_sm")

# sentence = "I hate abortion rights. Abortions should be banned."
# sentence_2 = "I like abortion rights. I belive we should keep them."
# sentence_3 = "I hate tennis. People should play tennis more often"

# ### STANCE SCORING ###

# # TODOs: https://www.cs.uic.edu/~liub/FBS/opinion-mining-final-WSDM.pdf 
# # TODOs: Pattern based Negation
# # TODOs: Semantic Orientation of an opinion (Claim)
# # TODOs:Group synonyms of 'features', 'targets'

# phrase_matcher = PhraseMatcher(nlp.vocab)

# ### SENTIMENT LEXICONS ###
# pos = [w.replace("\n", "") for w in open("../../data/lexicon/positive_lex.txt")]
# neg = [w.replace("\n", "") for w in open("../../data/lexicon/negative_lex.txt")]
# polarity_shifters = [w.replace("\n", "") for w in open("../../data/lexicon/shifter_lexicon.txt")]

# ### STANCE: ASPECT-SEMANTIC ORIENTATION ###
# def extract_aspect(sentence, n_gram):
#     aspects = extract_keyphrase(str(sentence))[0]

#     return nlp(aspects)

# def index_aspect(phrase, aspect, sentence):    
#     patterns = [nlp(aspect)]
#     phrase_matcher.add(phrase, None, *patterns)

#     start = 0
#     stop = 0

#     matched_phrases = phrase_matcher(sentence)
#     for i in matched_phrases:
#         _, start, stop = i
        
#     return start, stop

# # TODOs: Implement Polarity Shifters, Simple
# # TODOs: Implement Polarity Shifters, Complex, Verb Patterns
# def stance_score(start, stop, sentence):
#     pos_score = 0.0
#     neg_score = 0.0

#     score = 0
#     for idx, tok in enumerate(sentence):
#         if idx == start or idx == stop:
#             continue

#         # TODOs: Implement Polarity Shift
#         # TODOs: Experiement with descriptive term + keyphrase aspects
#         # TODOs: ABSA https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis
#         # Use external libaray: Textblob
        
#         k = 8
#         # Negation Rules
#         shifted_tok = None
#         shifted_toks = []

#         if (tok.dep_ == "neg") or (tok.dep_ in polarity_shifters):
#             #Shift to Negative
#             if idx <= k:
#                 if idx < start: neg_score += 1/(start - idx)
#                 else: neg_score += 1/(idx - stop)**0.5

#             if shifted_tok != None and shifted_tok in neg:
#                 print(shifted_tok.text)
#                 # Shift to Positive
#                 if idx < start: pos_score += 1/(start - idx)
#                 elif idx > start: pos_score += 1/(idx - stop)**0.5
#                 else: continue

#         # Aspect Sentement Orientation
#         if tok.text in pos:
#             if tok in shifted_toks:
#                 continue
            
#             if idx < start: pos_score += 1/(start - idx)
#             else: pos_score += 1/(idx - stop)**0.5

#         if tok.text in neg:
#             if tok in shifted_toks:
#                 continue

#             if idx <= start: neg_score += 1/(start - idx)
#             else: neg_score += 1/(idx - stop)**0.5
    
#     score = pos_score - neg_score /(pos_score + neg_score + 1)

#     return score

# def overlap_score(evidence_kp, adu_kp):
#     score = 0
    
#     # Split Keyphrase into components, scoring partial units as overlap
#     for i in evidence_kp:
#         for j in i.split():
#             # Ensure string value, to enact .find
#             if " ".join(adu_kp).find(j) != -1: 
#                 score += 1
#                 token = j
            
#             else: continue
    
#     return score

# def get_overlapping_token(evidence_kp, adu_kp):
#     for i in evidence_kp:
#         overlap_tokens = []
#         for j in i.split():
#             if " ".join(adu_kp).find(j) != -1: 
#                 overlap_tokens.append(j) 
            
#         return " ".join(i for i in overlap_tokens)

# def sentence_stance(sentence, aspect):
#     sentence = nlp(sentence)

#     start, stop = index_aspect("aspects", aspect, sentence)
#     score = stance_score(start, stop, sentence)

#     # Add Neutral
#     #stance = {"claim": sentence, "stance": "PRO", "aspect": aspect} if score > 0 else {"claim": sentence, "stance": "CON", "aspect": aspect}
    
#     return "PRO" if score > 0 else "CON"

# def fuzzy_match(target, evidence_unit):

#     overlapping_aspect = process.extractOne(target, ev.split())[0]
#     score = overlapping_aspect[1]

#     overlapping_aspect = nlp(re.sub(r'[^\w]', ' ', overlapping_aspect))

#     return overlapping_aspect, score

# def compare_stance(ev_unit, evidence_aspect, adu_target):
#     # Note: Already identified mathcing or partially matching Aspects. 

#     # Get the overlapping evidence aspect-target.
#     overlapping_target, score = fuzzy_match(target=adu_aspect, evidence_unit=ev)
    
#     # Get position of the overlapping_target
#     start, stop = index_aspect("OVERLAP", nlp(overlapping_target), nlp(ev_unit))

#     # Assert Stance towards evidence aspect
#     score = stance_score(start, stop, nlp(ev_unit))
    
#     return "PRO" if score > 0 else "CON"

# ev = "These simple ideas and techniques could help both you and your lover enjoy sex. 1 / 10 Getty Images/Caiaimage Think beyond the thrust."
# ev_aspect = "sex", "relationship", "opportunity"

# adu = 'Hello! Let me preface by saying I dont believe there is a better sex.'
# adu_aspect = "better sex"

# print(sentence_stance("The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're not in a relationship.", adu_aspect))
# print(compare_stance(ev, ev_aspect, adu_aspect))


In [ ]:
# from spacy.matcher import DependencyMatcher, Matcher
# matcher = Matcher(vocab=nlp.vocab)
# matcher

# # Matching Rule: Pronouns with Verbs that follow them
# aspect = "better sex"
# patterns = [
#     [{"DEP": "neg"}, {"LOWER": aspect}],
#     [{"DEP": "neg"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"POS": "VERB"}, {"POS": "ADJ"}, {"LOWER": aspect}],
#     [{"LOWER": aspect.lower()}]
# ]

# test = nlp("Hello! Let me preface by saying I dont believe there is a not better sex.")
# test_2 = nlp("These simple ideas and techniques could help both you and your lover enjoy better sex.")

# matcher.add("test", patterns=patterns)
# result = matcher(test_2, as_spans=True)

# result

# # for tok in test:
# #     print(tok.i, tok, tok.pos_, tok.dep_, tok.head.i, sep="\t")

In [ ]:
### TARGETED RETRIEVAL: ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)